## Efficient Simulation in the Inventory Model


#### John Stachurski

Simulation can be time consuming, especially if we need to simulate many paths

And it's often true that large sample sizes are necessary to get an accurate picture

Let's look at speeding up our simulations using [Numba](https://numba.pydata.org/)

In [2]:
import numpy as np
from numba import jit, njit, prange

Parameters:

In [3]:
s, S = 10, 100
p = 0.4
initial_x = 50.0

### Simulating One Path

Let's look at simulating one path with minimal optimization.

(There's at least some efficiency gain over pure Python because we'll use a vectorized function to generate the random variables.)

In [4]:
def sim_one_inventory_path(sim_length=10_000_000):
        
    dvals = np.random.geometric(p, size=sim_length-1) - 1
    X = np.empty(sim_length, dtype=np.int64)
    X[0] = initial_x
    
    for t, d in enumerate(dvals):
        x = X[t]
        if x <= s:
            X[t+1] = max(S - d, 0)
        else:
            X[t+1] = max(x - d, 0)
            
    return X

How fast does this run?

In [5]:
%%time

X = sim_one_inventory_path()

CPU times: user 8.97 s, sys: 156 ms, total: 9.13 s
Wall time: 9.23 s


Now let's build a jitted version.

In [6]:
@jit(nopython=True)
def update(x):        
    d = np.random.geometric(p) - 1
    if x <= s:
        y = np.maximum(S - d, 0)
    else:
        y = np.maximum(x - d, 0)
    return y

In [7]:
update(20)

16

In [8]:
@jit(nopython=True)
def sim_one_path_jitted(sim_length=10_000_000):
    X = np.empty(sim_length, dtype=np.int64)
    X[0] = initial_x
    for t in range(sim_length-1):
        X[t+1] = update(X[t])

This is **much** faster:

In [9]:
%%time

X = sim_one_path_jitted()

CPU times: user 472 ms, sys: 23 ms, total: 495 ms
Wall time: 495 ms


In [10]:
%%time

X = sim_one_path_jitted()

CPU times: user 391 ms, sys: 14.3 ms, total: 406 ms
Wall time: 409 ms


### Task: Calculate Stationary Order Rate

As we've seen, the inventory model has a stationary distribution.

Let's call it $\psi^*$, with the corresponding random variable denoted $X^*$.

For a large population with independent demand shocks,

$$ \mathbb P\{X^* = k\} \simeq \text{fraction of firms with inventory } k $$

We are interested in calculating

$$ \mathbb P\{X^* \leq s\} \simeq \text{fraction of firms currently placing orders } $$

The law of large numbers for stationary and ergodic sequences tells us that

$$ \mathbb P\{X^* \leq s\} \simeq \frac{1}{n} \sum_{t=1}^n \mathbb 1\{X_t \leq s\} $$

In other words, we can calculate the stationary probability $\mathbb P\{X^* \leq s\}$ by generating a single time series and calculating the fraction of time that it spends below $s$.

We can do this fast with the jitted code above, thanks to Numba.

#### Exploiting parallelization

However, the above approach is difficult to parallelize because it's inherently sequential

An approach that's easier to parallelize is to 

* pick some large value $T$

* generate many independent observations $X^i_T$ of $X_T$

* calculate the fraction of observations that are less than $s$

This works because 

$$ \mathbb P\{X^* \leq s\} \simeq \mathbb P \{X_T \leq s\} $$

when $T$ is large (since, as previously discussed, $\psi_t \to \psi^*$).

Thus, letting $m$ be the number of paths we'll generate, our goal is to fix large $T$ and compute

$$ p(m, T) = \frac{1}{m} \sum_{i=1}^m \mathbb 1\{X_T^i \leq s\} $$

### Simulating Many Paths

Here's a jitted (and hence fast) piece of code for simulating many paths.

In [11]:
@njit
def sim_prob(m=500_000, T=1000):
    
    is_below_s = 0
    for i in range(m):
        x = initial_x
        for t in range(T):
            x = update(x)
        if x <= s:
            is_below_s += 1
    p_m_T = is_below_s / m
    return p_m_T


Let's get a reading on the time.

In [12]:
%%time

p = sim_prob()


CPU times: user 18.6 s, sys: 62 ms, total: 18.6 s
Wall time: 18.6 s


In [13]:
p

0.01637

Now let's try a simple parallelization:

In [14]:
@njit(parallel=True)
def sim_prob_parallel(m=500_000, T=1000):
    
    is_below_s = 0
    for i in prange(m):
        x = initial_x
        for t in range(T):
            x = update(x)
        if x <= s:
            is_below_s += 1
    p_m_T = is_below_s / m
    return p_m_T


In [15]:
%%time

p = sim_prob_parallel()

CPU times: user 26.6 s, sys: 39.5 ms, total: 26.6 s
Wall time: 7.42 s


In [16]:
p

0.016322

Let's try for a more efficient parallelization, where the work is divided into a smaller number of chunks.

In [17]:
@njit(parallel=True)
def sim_prob_parallel(m=500_000, T=1000, chunks=500):
    
    chunk_size = m // chunks
    p_m_T_vals = np.empty(chunks)
    
    for i in prange(chunks):
        p_m_T_vals[i] = sim_prob(m=chunk_size)

    return p_m_T_vals.mean()

In [18]:
%%time

X = sim_prob_parallel()


CPU times: user 26.5 s, sys: 104 ms, total: 26.6 s
Wall time: 7.92 s


There's essentially no difference --- so `prange` is already optimizing the load across CPUs for us!